In [1]:
import os

from tqdm import tqdm

import yaml
from yaml import CLoader as Loader

from munch import munchify

from vicreg import *

In [2]:
root_dir = '/home/minh/PycharmProjects/prostate_cancer_classification_v1'
with open(f'{root_dir}/yamls/vicreg_patch.yml') as f:
    args = yaml.load(f, Loader)
args = munchify(args)
print(args)

Munch({'root_dir': '/home/minh/PycharmProjects/prostate_cancer_classification_v1', 'exp_dir': 'experiments/vicreg_patch', 'log_freq_time': 60, 'arch': 'resnet50', 'mlp': '8192-8192-8192', 'epochs': 1, 'batch_size': 128, 'base_lr': 0.2, 'wd': '1e-6', 'sim_coef': 25.0, 'std_coef': 25.0, 'cov_coef': 1.0, 'num_workers': 10, 'device': 'cuda', 'world_size': 1, 'local_rank': -1, 'dist_url': 'env://'})


In [3]:
from utils_3d.dataset import PatchUnlabeledDataset
from augmentations import TwoCropsTransform

data_root = '/media/minh/My Passport/workspace/Preparation/DataPreparation/dummy/snapshots_in_use_Amoon_2''/patch_32x32_whole_prostate/'
dataset = PatchUnlabeledDataset(data_root, TwoCropsTransform(), pid_range=(0, 100))

In [4]:
torch.backends.cudnn.benchmark = True
# init_distributed_mode(args)
print(args)
gpu = torch.device(args.device)
args.exp_dir = '/'.join([args.root_dir, args.exp_dir])
os.makedirs(args.exp_dir, exist_ok=True)
stats_file = open('/'.join([args.exp_dir, "stats.txt"]), "a", buffering=1)
print(" ".join(sys.argv))
print(" ".join(sys.argv), file=stats_file)

per_device_batch_size = args.batch_size // args.world_size
loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=per_device_batch_size,
    num_workers=args.num_workers,
    pin_memory=False,
)
model = VICReg(args).cuda(gpu)
optimizer = LARS(
    model.parameters(),
    lr=0,
    weight_decay=args.wd,
    weight_decay_filter=exclude_bias_and_norm,
    lars_adaptation_filter=exclude_bias_and_norm,
)
if os.path.exists('/'.join([args.exp_dir, "model.pth"])):
    ckpt = torch.load('/'.join([args.exp_dir, "model.pth"]), map_location="cpu")
    start_epoch = ckpt["epoch"]
    model.load_state_dict(ckpt["model"])
    optimizer.load_state_dict(ckpt["optimizer"])
else:
    start_epoch = 0

Munch({'root_dir': '/home/minh/PycharmProjects/prostate_cancer_classification_v1', 'exp_dir': 'experiments/vicreg_patch', 'log_freq_time': 60, 'arch': 'resnet50', 'mlp': '8192-8192-8192', 'epochs': 1, 'batch_size': 128, 'base_lr': 0.2, 'wd': '1e-6', 'sim_coef': 25.0, 'std_coef': 25.0, 'cov_coef': 1.0, 'num_workers': 10, 'device': 'cuda', 'world_size': 1, 'local_rank': -1, 'dist_url': 'env://'})
/home/minh/anaconda3/envs/torch/lib/python3.8/site-packages/ipykernel_launcher.py -f /home/minh/.local/share/jupyter/runtime/kernel-dd7b7588-497e-49f6-9b65-66f2d053c469.json


In [5]:
start_time = last_logging = time.time()
scaler = torch.cuda.amp.GradScaler()
for epoch in range(start_epoch, args.epochs):
    for step, (x, y) in enumerate(loader, start=epoch * len(loader)):
        x = x.cuda(gpu, non_blocking=True)
        y = y.cuda(gpu, non_blocking=True)
        lr = adjust_learning_rate(args, optimizer, loader, step)

        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            loss = model.forward(x, y)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        break

        current_time = time.time()
        if current_time - last_logging > args.log_freq_time:
            stats = dict(
                epoch=epoch,
                step=step,
                loss=loss.item(),
                time=int(current_time - start_time),
                lr=lr,
            )
            print(json.dumps(stats))
            print(json.dumps(stats), file=stats_file)
            last_logging = current_time
    state = dict(
        epoch=epoch + 1,
        model=model.state_dict(),
        optimizer=optimizer.state_dict(),
    )
    torch.save(state, args.exp_dir / "model.pth")

torch.save(model.module.backbone.state_dict(), args.exp_dir / "resnet50.pth")

/home/minh/anaconda3/envs/torch/lib/python3.8/site-packages/torch/autograd/__init__.py:173: UserWarning: Error detected in VarBackward0. Traceback of forward call that caused the error:
  File "/home/minh/anaconda3/envs/torch/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/minh/anaconda3/envs/torch/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/home/minh/anaconda3/envs/torch/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/home/minh/anaconda3/envs/torch/lib/python3.8/site-packages/traitlets/config/application.py", line 846, in launch_instance
    app.start()
  File "/home/minh/anaconda3/envs/torch/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/home/minh/anaconda3/envs/torch/lib/python3.8/site-packages/tornado/platform/asyncio.py", line 215, in start
    self.a

RuntimeError: Function 'VarBackward0' returned nan values in its 0th output.

In [ ]:
model.backbone